# Cross Validation

## k-fold Cross Validation

In [ ]:
import numpy as np
from sklearn.model_selection import KFold

X = np.array([[101, 202], [303, 404], [102, 204], [306, 408]])
y = np.array([101, 202, 303, 404])

In [ ]:
X

array([[101, 202],
       [303, 404],
       [102, 204],
       [306, 408]])

In [ ]:
kf = KFold(n_splits=2)

In [ ]:
kf.split(X)

<generator object _BaseKFold.split at 0x186da7040>

In [ ]:
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [2 3] TEST: [0 1]
TRAIN: [0 1] TEST: [2 3]


## Leave-one-out

In [ ]:
from sklearn.model_selection import LeaveOneOut

X = np.array([[101, 202], [303, 404], [102, 204], [306, 408]])
y = np.array([101, 202, 303, 404])
loo = LeaveOneOut()

for train_index, test_index in loo.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [1 2 3] TEST: [0]
TRAIN: [0 2 3] TEST: [1]
TRAIN: [0 1 3] TEST: [2]
TRAIN: [0 1 2] TEST: [3]


## Stratified k-fold

In [ ]:
from sklearn.model_selection import StratifiedKFold

X = np.array([[101, 202], [102, 204], [303, 404], [306, 408]])
y = np.array([0, 0, 1, 1])
skf = StratifiedKFold(n_splits=2)

for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [1 3] TEST: [0 2]
TRAIN: [0 2] TEST: [1 3]


#  Пошук оптимальних гіперпараметрів

Завантажимо набір даних.

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


def get_indices(X):
    train_full_id, test_id = train_test_split(X.index, test_size=test_size, shuffle=True, random_state=42)
    train_id, val_id = train_test_split(train_full_id, test_size=test_size, shuffle=True, random_state=42)
    return train_id, val_id, test_id

def get_metrics(y_true, y_predicted, n_digits=5):
    params = dict(y_true=y_true, y_pred=y_predicted)
    mse = mean_squared_error(**params)
    rmse = mean_squared_error(**params, squared=False)
    return dict(mse=round(mse, n_digits), rmse=round(rmse, n_digits))

In [ ]:
dataset = fetch_california_housing()
df = pd.DataFrame(dataset['data'], columns = dataset['feature_names'])
target_col_name = dataset['target_names'][0]
df[target_col_name] = dataset['target']
del dataset

In [ ]:
X = df.drop([target_col_name],axis=1)
y = df[target_col_name]

In [ ]:
test_size = 0.2

train_id, val_id, test_id = get_indices(X)

## Grid search

In [ ]:
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
degree = 2
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly_features', PolynomialFeatures(degree=degree)),
    ('model', Lasso())
])

In [ ]:
np.arange(0,10,0.1).shape

(100,)

In [ ]:
search = GridSearchCV(
    pipeline,
    {'model__alpha':np.arange(0,10,0.1)},
    cv = 5, 
    scoring="neg_mean_squared_error",
    verbose=3
)

In [ ]:
search.fit(X.loc[train_id], y.loc[train_id])

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/Users/hanna.pylieva/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/pipeline.py:394: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/hanna.pylieva/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/Users/hanna.pylieva/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.155e+03, tolerance: 1.397e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of

[CV 1/5] END .................model__alpha=0.0;, score=-3.869 total time=   0.9s


/Users/hanna.pylieva/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.181e+03, tolerance: 1.395e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/hanna.pylieva/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/pipeline.py:394: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/hanna.pylieva/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: UserWarning: Coordinate descent wi

[CV 2/5] END ................model__alpha=0.0;, score=-56.554 total time=   0.9s


...

[CV 3/5] END ..model__alpha=2.3000000000000003;, score=-1.303 total time=   0.0s
[CV 4/5] END ..model__alpha=2.3000000000000003;, score=-1.364 total time=   0.0s
[CV 5/5] END ..model__alpha=2.3000000000000003;, score=-1.275 total time=   0.0s
[CV 1/5] END ..model__alpha=2.4000000000000004;, score=-1.340 total time=   0.0s
[CV 2/5] END ..model__alpha=2.4000000000000004;, score=-1.348 total time=   0.0s
[CV 3/5] END ..model__alpha=2.4000000000000004;, score=-1.303 total time=   0.0s
[CV 4/5] END ..model__alpha=2.4000000000000004;, score=-1.364 total time=   0.0s
[CV 5/5] END ..model__alpha=2.4000000000000004;, score=-1.275 total time=   0.0s
[CV 1/5] END .................model__alpha=2.5;, score=-1.340 total time=   0.0s
[CV 2/5] END .................model__alpha=2.5;, score=-1.348 total time=   0.0s
[CV 3/5] END .................model__alpha=2.5;, score=-1.303 total time=   0.0s
[CV 4/5] END .................model__alpha=2.5;, score=-1.364 total time=   0.0s
[CV 5/5] END ...............

[CV 1/5] END ..model__alpha=4.6000000000000005;, score=-1.340 total time=   0.0s
[CV 2/5] END ..model__alpha=4.6000000000000005;, score=-1.348 total time=   0.0s
[CV 3/5] END ..model__alpha=4.6000000000000005;, score=-1.303 total time=   0.0s
[CV 4/5] END ..model__alpha=4.6000000000000005;, score=-1.364 total time=   0.0s
[CV 5/5] END ..model__alpha=4.6000000000000005;, score=-1.275 total time=   0.0s
[CV 1/5] END .................model__alpha=4.7;, score=-1.340 total time=   0.0s
[CV 2/5] END .................model__alpha=4.7;, score=-1.348 total time=   0.0s
[CV 3/5] END .................model__alpha=4.7;, score=-1.303 total time=   0.0s
[CV 4/5] END .................model__alpha=4.7;, score=-1.364 total time=   0.0s
[CV 5/5] END .................model__alpha=4.7;, score=-1.275 total time=   0.0s
[CV 1/5] END ...model__alpha=4.800000000000001;, score=-1.340 total time=   0.0s
[CV 2/5] END ...model__alpha=4.800000000000001;, score=-1.348 total time=   0.0s
[CV 3/5] END ...model__alpha

[CV 3/5] END ...model__alpha=6.800000000000001;, score=-1.303 total time=   0.0s
[CV 4/5] END ...model__alpha=6.800000000000001;, score=-1.364 total time=   0.0s
[CV 5/5] END ...model__alpha=6.800000000000001;, score=-1.275 total time=   0.0s
[CV 1/5] END .................model__alpha=6.9;, score=-1.340 total time=   0.0s
[CV 2/5] END .................model__alpha=6.9;, score=-1.348 total time=   0.0s
[CV 3/5] END .................model__alpha=6.9;, score=-1.303 total time=   0.0s
[CV 4/5] END .................model__alpha=6.9;, score=-1.364 total time=   0.0s
[CV 5/5] END .................model__alpha=6.9;, score=-1.275 total time=   0.0s
[CV 1/5] END .................model__alpha=7.0;, score=-1.340 total time=   0.0s
[CV 2/5] END .................model__alpha=7.0;, score=-1.348 total time=   0.0s
[CV 3/5] END .................model__alpha=7.0;, score=-1.303 total time=   0.0s
[CV 4/5] END .................model__alpha=7.0;, score=-1.364 total time=   0.0s
[CV 5/5] END ...............

[CV 2/5] END .................model__alpha=9.0;, score=-1.348 total time=   0.0s
[CV 3/5] END .................model__alpha=9.0;, score=-1.303 total time=   0.0s
[CV 4/5] END .................model__alpha=9.0;, score=-1.364 total time=   0.0s
[CV 5/5] END .................model__alpha=9.0;, score=-1.275 total time=   0.0s
[CV 1/5] END .................model__alpha=9.1;, score=-1.340 total time=   0.0s
[CV 2/5] END .................model__alpha=9.1;, score=-1.348 total time=   0.0s
[CV 3/5] END .................model__alpha=9.1;, score=-1.303 total time=   0.0s
[CV 4/5] END .................model__alpha=9.1;, score=-1.364 total time=   0.0s
[CV 5/5] END .................model__alpha=9.1;, score=-1.275 total time=   0.0s
[CV 1/5] END ...model__alpha=9.200000000000001;, score=-1.340 total time=   0.0s
[CV 2/5] END ...model__alpha=9.200000000000001;, score=-1.348 total time=   0.0s
[CV 3/5] END ...model__alpha=9.200000000000001;, score=-1.303 total time=   0.0s
[CV 4/5] END ...model__alpha

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('poly_features', PolynomialFeatures()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5,
       2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8,
       3.9, 4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1,
       5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4,
       6.5, 6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7,
       7.8, 7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. ,
       9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='neg_mean_squared_error', verbose=3)

In [ ]:
search.best_params_

{'model__alpha': 0.1}

In [ ]:
search.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('poly_features', PolynomialFeatures()),
                ('model', Lasso(alpha=0.1))])

In [ ]:
coefficients = search.best_estimator_.named_steps['model'].coef_

Бачимо, що серед коефіцієнтів у нас вийшло багато нулів і за фактом значущими залишилося лише кілька.

In [ ]:
coefficients.round(2)

array([ 0.  ,  0.71,  0.1 , -0.  , -0.  , -0.  , -0.  , -0.  , -0.  ,
       -0.  ,  0.  ,  0.  , -0.  , -0.  , -0.  ,  0.  , -0.  ,  0.  ,
        0.  ,  0.  ,  0.  , -0.  ,  0.  , -0.  ,  0.  , -0.  ,  0.  ,
        0.  , -0.  , -0.  , -0.  ,  0.  ,  0.  , -0.  , -0.  , -0.  ,
        0.  ,  0.  , -0.  , -0.  , -0.  , -0.  , -0.02,  0.  ,  0.  ])

Давайте дізнаємося, що за коефіцієнти найзначніші:

In [ ]:
# dir(search.best_estimator_.named_steps['poly_features'])

In [ ]:
feature_names = search.best_estimator_.named_steps['poly_features'].get_feature_names_out()
coef_df = pd.DataFrame(zip(feature_names, coefficients.round(3)), columns=['feature_name', 'coef'])

In [ ]:
coef_df.sort_values('coef', ascending=False)[:10]

,feature_name,coef
1,x0,0.706
2,x1,0.105
33,x3 x6,-0.000
24,x2^2,0.000
25,x2 x3,-0.000
26,x2 x4,0.000
27,x2 x5,0.000
28,x2 x6,-0.000
29,x2 x7,-0.000
30,x3^2,-0.000


Словник з назвами ознак:

In [ ]:
X.columns.to_frame().reset_index(drop=True).to_dict()[0]

{0: 'MedInc',
 1: 'HouseAge',
 2: 'AveRooms',
 3: 'AveBedrms',
 4: 'Population',
 5: 'AveOccup',
 6: 'Latitude',
 7: 'Longitude'}

Бачимо, що все одно найважливіша ознака – MedInc без жодних ступенів.

In [ ]:
search.best_score_

-0.6640858768504454

In [ ]:
best_model = search.best_estimator_

y_train_pred = best_model.predict(X.loc[train_id])
y_val_pred = best_model.predict(X.loc[val_id])
train_metrics = get_metrics(y.loc[train_id], y_train_pred)
val_metrics = get_metrics(y.loc[val_id], y_val_pred)

In [ ]:
train_metrics, val_metrics

({'mse': 0.66241, 'rmse': 0.81389}, {'mse': 0.69114, 'rmse': 0.83135})

## Random Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
params = dict()

# значения для alpha: 100 значений мужду e^-5 и e^5
params['alpha'] =  np.logspace(-5, 5, 100, endpoint=True)

# значения для l1_ratio: 100 значений между 0 и 1
params['l1_ratio'] = np.arange(0, 1, 0.01)

In [ ]:
len(params['alpha'])*len(params['l1_ratio'])

10000

In [ ]:
model = ElasticNet()

In [ ]:
search = RandomizedSearchCV(
    model,
    params,
    n_iter = 100,
    cv=5,
    scoring="neg_mean_squared_error",
    verbose=3,
    refit=True
)

In [ ]:
search.fit(X.loc[train_id], y.loc[train_id])

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5] END alpha=2.2570197196339215, l1_ratio=0.47000000000000003;, score=-1.074 total time=   0.0s
[CV 2/5] END alpha=2.2570197196339215, l1_ratio=0.47000000000000003;, score=-1.079 total time=   0.0s
[CV 3/5] END alpha=2.2570197196339215, l1_ratio=0.47000000000000003;, score=-1.036 total time=   0.0s
[CV 4/5] END alpha=2.2570197196339215, l1_ratio=0.47000000000000003;, score=-1.098 total time=   0.0s
[CV 5/5] END alpha=2.2570197196339215, l1_ratio=0.47000000000000003;, score=-1.012 total time=   0.0s
[CV 1/5] END alpha=0.0042292428743894986, l1_ratio=0.52;, score=-0.524 total time=   0.0s
[CV 2/5] END alpha=0.0042292428743894986, l1_ratio=0.52;, score=-0.509 total time=   0.0s
[CV 3/5] END alpha=0.0042292428743894986, l1_ratio=0.52;, score=-0.501 total time=   0.1s
[CV 4/5] END alpha=0.0042292428743894986, l1_ratio=0.52;, score=-0.529 total time=   0.0s
[CV 5/5] END alpha=0.0042292428743894986, l1_ratio=0.52;, score=-0.

[CV 5/5] END alpha=7.220809018385457, l1_ratio=0.8300000000000001;, score=-1.274 total time=   0.0s
[CV 1/5] END alpha=0.010722672220103232, l1_ratio=0.39;, score=-0.522 total time=   0.0s
[CV 2/5] END alpha=0.010722672220103232, l1_ratio=0.39;, score=-0.512 total time=   0.0s
[CV 3/5] END alpha=0.010722672220103232, l1_ratio=0.39;, score=-0.505 total time=   0.1s
[CV 4/5] END alpha=0.010722672220103232, l1_ratio=0.39;, score=-0.529 total time=   0.0s
[CV 5/5] END alpha=0.010722672220103232, l1_ratio=0.39;, score=-0.523 total time=   0.0s
[CV 1/5] END .......alpha=1e-05, l1_ratio=0.14;, score=-0.527 total time=   0.0s
[CV 2/5] END .......alpha=1e-05, l1_ratio=0.14;, score=-0.508 total time=   0.0s
[CV 3/5] END .......alpha=1e-05, l1_ratio=0.14;, score=-0.499 total time=   0.0s
[CV 4/5] END .......alpha=1e-05, l1_ratio=0.14;, score=-0.529 total time=   0.0s
[CV 5/5] END .......alpha=1e-05, l1_ratio=0.14;, score=-0.518 total time=   0.1s
[CV 1/5] END alpha=756.463327554629, l1_ratio=0.74

[CV 5/5] END alpha=0.0042292428743894986, l1_ratio=0.64;, score=-0.520 total time=   0.1s
[CV 1/5] END alpha=117.68119524349991, l1_ratio=0.93;, score=-1.340 total time=   0.0s
[CV 2/5] END alpha=117.68119524349991, l1_ratio=0.93;, score=-1.348 total time=   0.0s
[CV 3/5] END alpha=117.68119524349991, l1_ratio=0.93;, score=-1.303 total time=   0.0s
[CV 4/5] END alpha=117.68119524349991, l1_ratio=0.93;, score=-1.364 total time=   0.0s
[CV 5/5] END alpha=117.68119524349991, l1_ratio=0.93;, score=-1.275 total time=   0.0s
[CV 1/5] END alpha=0.10974987654930568, l1_ratio=0.8200000000000001;, score=-0.596 total time=   0.0s
[CV 2/5] END alpha=0.10974987654930568, l1_ratio=0.8200000000000001;, score=-0.580 total time=   0.0s
[CV 3/5] END alpha=0.10974987654930568, l1_ratio=0.8200000000000001;, score=-0.580 total time=   0.0s
[CV 4/5] END alpha=0.10974987654930568, l1_ratio=0.8200000000000001;, score=-0.596 total time=   0.0s
[CV 5/5] END alpha=0.10974987654930568, l1_ratio=0.8200000000000001

[CV 2/5] END alpha=0.00025950242113997375, l1_ratio=0.87;, score=-0.508 total time=   0.0s
[CV 3/5] END alpha=0.00025950242113997375, l1_ratio=0.87;, score=-0.499 total time=   0.0s
[CV 4/5] END alpha=0.00025950242113997375, l1_ratio=0.87;, score=-0.529 total time=   0.0s
[CV 5/5] END alpha=0.00025950242113997375, l1_ratio=0.87;, score=-0.518 total time=   0.0s
[CV 1/5] END alpha=0.0042292428743894986, l1_ratio=0.16;, score=-0.524 total time=   0.1s
[CV 2/5] END alpha=0.0042292428743894986, l1_ratio=0.16;, score=-0.509 total time=   0.0s
[CV 3/5] END alpha=0.0042292428743894986, l1_ratio=0.16;, score=-0.501 total time=   0.0s
[CV 4/5] END alpha=0.0042292428743894986, l1_ratio=0.16;, score=-0.529 total time=   0.0s
[CV 5/5] END alpha=0.0042292428743894986, l1_ratio=0.16;, score=-0.519 total time=   0.0s
[CV 1/5] END alpha=0.1747528400007683, l1_ratio=0.75;, score=-0.651 total time=   0.0s
[CV 2/5] END alpha=0.1747528400007683, l1_ratio=0.75;, score=-0.635 total time=   0.0s
[CV 3/5] END

[CV 4/5] END alpha=0.04328761281083057, l1_ratio=0.71;, score=-0.548 total time=   0.0s
[CV 5/5] END alpha=0.04328761281083057, l1_ratio=0.71;, score=-0.547 total time=   0.0s
[CV 1/5] END alpha=6.428073117284319e-05, l1_ratio=0.55;, score=-0.527 total time=   0.0s
[CV 2/5] END alpha=6.428073117284319e-05, l1_ratio=0.55;, score=-0.508 total time=   0.0s
[CV 3/5] END alpha=6.428073117284319e-05, l1_ratio=0.55;, score=-0.499 total time=   0.1s
[CV 4/5] END alpha=6.428073117284319e-05, l1_ratio=0.55;, score=-0.529 total time=   0.2s
[CV 5/5] END alpha=6.428073117284319e-05, l1_ratio=0.55;, score=-0.518 total time=   0.2s
[CV 1/5] END alpha=298.364724028334, l1_ratio=0.02;, score=-1.341 total time=   0.0s
[CV 2/5] END alpha=298.364724028334, l1_ratio=0.02;, score=-1.347 total time=   0.0s
[CV 3/5] END alpha=298.364724028334, l1_ratio=0.02;, score=-1.300 total time=   0.0s
[CV 4/5] END alpha=298.364724028334, l1_ratio=0.02;, score=-1.364 total time=   0.0s
[CV 5/5] END alpha=298.36472402833

...

/Users/hanna.pylieva/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.729e+03, tolerance: 1.407e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


[CV 3/5] END ........alpha=1e-05, l1_ratio=0.0;, score=-0.499 total time=   0.6s


/Users/hanna.pylieva/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.689e+03, tolerance: 1.391e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/hanna.pylieva/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.707e+03, tolerance: 1.415e+00 Linear regression models with null weight for the l1 regularization term are more efficiently 

[CV 4/5] END ........alpha=1e-05, l1_ratio=0.0;, score=-0.529 total time=   0.9s
[CV 5/5] END ........alpha=1e-05, l1_ratio=0.0;, score=-0.518 total time=   0.2s
[CV 1/5] END alpha=2420.1282647943835, l1_ratio=0.3;, score=-1.340 total time=   0.0s
[CV 2/5] END alpha=2420.1282647943835, l1_ratio=0.3;, score=-1.348 total time=   0.0s
[CV 3/5] END alpha=2420.1282647943835, l1_ratio=0.3;, score=-1.303 total time=   0.0s
[CV 4/5] END alpha=2420.1282647943835, l1_ratio=0.3;, score=-1.364 total time=   0.0s
[CV 5/5] END alpha=2420.1282647943835, l1_ratio=0.3;, score=-1.275 total time=   0.0s
[CV 1/5] END alpha=0.04328761281083057, l1_ratio=0.6;, score=-0.538 total time=   0.0s
[CV 2/5] END alpha=0.04328761281083057, l1_ratio=0.6;, score=-0.533 total time=   0.0s
[CV 3/5] END alpha=0.04328761281083057, l1_ratio=0.6;, score=-0.530 total time=   0.0s
[CV 4/5] END alpha=0.04328761281083057, l1_ratio=0.6;, score=-0.547 total time=   0.0s
[CV 5/5] END alpha=0.04328761281083057, l1_ratio=0.6;, score

RandomizedSearchCV(cv=5, estimator=ElasticNet(), n_iter=100,
                   param_distributions={'alpha': array([1.00000000e-05, 1.26185688e-05, 1.59228279e-05, 2.00923300e-05,
       2.53536449e-05, 3.19926714e-05, 4.03701726e-05, 5.09413801e-05,
       6.42807312e-05, 8.11130831e-05, 1.02353102e-04, 1.29154967e-04,
       1.62975083e-04, 2.05651231e-04, 2.59502421e-04, 3.27454916e-04,
       4.13201240e-04, 5...
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99])},
                   scoring='neg_mean_squared_error', verbose=3)

In [ ]:
display(search.best_estimator_, search.best_score_)

ElasticNet(alpha=0.00210490414451202, l1_ratio=0.3)

-0.5162285995272566

In [ ]:
ElasticNet()

In [ ]:
best_model = search.best_estimator_

y_train_pred = best_model.predict(X.loc[train_id])
y_val_pred = best_model.predict(X.loc[val_id])
train_metrics = get_metrics(y.loc[train_id], y_train_pred)
val_metrics = get_metrics(y.loc[val_id], y_val_pred)

In [ ]:
train_metrics, val_metrics

({'mse': 0.51304, 'rmse': 0.71627}, {'mse': 0.53884, 'rmse': 0.73406})

Вийшло якісно покращити модель.